In [14]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [15]:
data = pd.read_csv('data_train_test.csv')
a = data['patient_manual_class']
a = a.replace(np.nan, 0)
a = a.replace(2, 0)
data['patient_manual_class'] = a
data = data.dropna()

In [16]:
from sklearn.model_selection import train_test_split
num_features = ['abs', 'std', 'norm_abs', 'mean', 'sd', 'median', 'mode', 'Q25', 'Q75', 'IQR', 'skew', 'kurt', 'kx2', 'n_pic']
cat_features = ['patient_ds', 'rec_date_time']
X_train, X_test, y_train, y_test = train_test_split(data[num_features], data['patient_manual_class'], test_size=0.2, shuffle=False)

In [17]:
df = pd.DataFrame(data[num_features])

In [18]:
model_LR = LogisticRegression(solver='liblinear', multi_class='ovr').fit(X_train, y_train)

In [19]:
import pickle
data = model_LR
with open('model_LR.pickle', 'wb') as f:
  pickle.dump(data, f)

In [20]:
prediction = model_LR.predict(X_test)
from sklearn import metrics
print("Точность:", metrics.accuracy_score(y_test, prediction))

Точность: 0.8222222222222222


In [21]:
from sklearn.metrics import mean_squared_error
print("Ошибка MSE:", mean_squared_error(y_test, prediction))

Ошибка MSE: 0.17777777777777778


In [22]:
from math import sqrt
print("Ошибка RMSE:", sqrt(mean_squared_error(y_test, prediction)))

Ошибка RMSE: 0.4216370213557839


In [23]:
data_test = pd.read_csv('data_test.csv')
df_test = pd.DataFrame(data_test[num_features])
df_test = df_test.dropna()
X_test = df_test.iloc[:,:]
prediction = model_LR.predict(X_test)
df_test["fistulas_kind"] = prediction #добавляем в наш НД расчетное значение
#пишем в файл результат, колонка fistulas_kind:
df_test.to_csv("data_test_model_LR.csv", index=False)
print("Результаты классификации модели, построенной на логистической регрессии выгружены в файл data_test_model_LR.csv")

Результаты классификации модели, построенной на логистической регрессии выгружены в файл data_test_model_LR.csv


In [24]:
print("Виды АВФ в результате классификации с помощью логистической регрессии: ", df_test.fistulas_kind.unique())
print("0 - плохая, 1 - хорошая")
print("\n", df_test.groupby('fistulas_kind').size())

Виды АВФ в результате классификации с помощью логистической регрессии:  [1. 0.]
0 - плохая, 1 - хорошая

 fistulas_kind
0.0      4
1.0    230
dtype: int64
